In [1]:
import tensorflow as tf

import numpy as np

train = np.array([*range(1, 11)], dtype=np.float32)

label = train * 0.5 + 0.5

In [ ]:
!apt-get install tensorflow-model-server

In [3]:
train

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.], dtype=float32)

In [4]:
label

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5], dtype=float32)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(1, input_shape=(1,))
])

model.compile(optimizer='sgd', loss='mse')

model.fit(train, label, epochs=1000, verbose=0)

In [ ]:
import os

model_folder = "/content/linear"

model_version = "1"

model.export(os.path.join(model_folder, model_version))

In [7]:
os.environ["MODEL_FOLDER"] = model_folder

In [8]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=linear_model \
  --model_base_path="${MODEL_FOLDER}" > server.log 2>&1

In [ ]:
!tail server.log

In [10]:
localurl = "http://localhost:8501/v1/models/linear_model:predict"

os.environ["localurl"] = localurl

In [ ]:
!curl -d "{\"instances\": [[11]]}" -H 'content-type: application/json' -X POST $localurl

```shell
{
  "predictions": [[6.00248289]]
}
```

In [ ]:
!curl -d "{\"instances\": [[11], [12]]}" -H 'content-type: application/json' -X POST $localurl

```shell
{
  "predictions": [[6.00248289], [6.50309801]]
}
```

In [13]:
import json, requests

content = {"content-type": "application/json"}

data = json.dumps({"instances":[[11], [12]]})

response = requests.post(localurl, data=data, headers=content)

response.text

'{\n    "predictions": [[6.00248289], [6.50309801]\n    ]\n}'

In [14]:
result = json.loads(response.text)

result = result["predictions"]

result

[[6.00248289], [6.50309801]]

In [15]:
data = json.dumps({"instances":[[11]]})

response = requests.post(localurl, data=data, headers=content)

result = json.loads(response.text)

result = result["predictions"]

result

[[6.00248289]]